In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from sqlalchemy.sql import and_, or_, not_
import re

In [ ]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)
events = Table('events', metadata, autoload = True, autoload_with = engine)

In [ ]:
stmt = select([games.c.id]).where(games.columns.isProcessed == True)
results = conn.execute(stmt).fetchall()
gameIDs = [value for value, in results]
len(gameIDs)

In [ ]:
def cleanAction(x):
    x = re.sub('^\d+', '', x).lstrip()
    x = re.sub('\d+$', '', x).rstrip()
    x = re.sub(' by$', '', x).rstrip()
    return x

In [ ]:
actValMap = {
'Assist by':0,
'Block by':0,
'Foul by':0,
'Steal by':0,
'Technicaloul by':0,
'Turnover by':0,
'deadball rebound':0,
'defensive rebound':0,
'enters the game':0,
'goes to the bench':0,
'made 2-pt field goal':2,
'made 3-pt jump shot':3,
'made dunk':2,
'made free throw':1,
'made jump shot':2,
'made layup':2,
'made tip-in':2,
'missed 2-pt field goal':0,
'missed 3-pt jump shot':0,
'missed dunk':0,
'missed free throw':0,
'missed jump shot':0,
#missed jump shot3missed 3-pt jump shot2missed 2-pt field goalmissed layupmissed free throwmissed dunkmissed tip-in}	123
'missed layup':0,
'missed tip-in':0,
'offensive rebound':0,
'pointswrong basket by defense':-2,
#score=2text=made jump shot3score=3text=made 3-pt jump shot2score=2text=made 2-pt field goalscore=2text=made layupscore=1text=made free throwscore=2text=made dunkscore=2text=made tip-in	108
'will be starting':0
}

In [ ]:
stmt = "SELECT * FROM events WHERE gameid = '" + gameIDs[0] + "'"
n1 = 0
    
df = pd.read_sql_query(stmt,engine)
df['scoreHome'] = df['scoreHome'].map(int)
df['scoreAway'] = df['scoreAway'].map(int)

df['action'] = df['action'].apply(cleanAction)

df['actionValue'] = df['action'].map(actValMap).map(int,na_action='ignore')

conditions = [
    (df['actionValue'] == 1),
    (df['actionValue'] == 2),
    (df['actionValue'] == 3)
]
choices = ['FTM', 'FG2', 'FG3']

df['action_edit1'] = np.select(conditions, choices, default=df['action'])
#df[df['action_edit1']==0]['action_edit1'] = df[df['action_edit1']==0]['action']

#df.groupby('player').aggregate({'playScore':'sum'
#                               ,})
df.head(40)

In [ ]:
stmt = "SELECT * FROM events"
conn.execute(stmt).fetchall()

In [ ]:
#view all actions and frequency
#"""
stmt = "\
SELECT \
action\
,COUNT(1) \
FROM events \
WHERE action NOT LIKE '%time%out%'\
GROUP BY \
action \
ORDER BY 2 desc"

df = pd.read_sql_query(stmt,engine)
#"""

df['action'] = df['action'].apply(cleanAction)

df['actionValue'] = df['action'].map(actValMap)

df = df.groupby('action').sum()

df.head(50)

In [ ]:
stmt = "SELECT  \
e.gameid\
,e.period\
,e.time\
,e.action AS action1 \
,e2.action AS action2 \
,e.team AS team1 \
,e2.team AS team2 \
FROM events AS e \
INNER JOIN events AS e2 \
ON e.gameid = e2.gameid \
AND e.period = e2.period \
AND e.time = e2.time \
AND e.action != e2.action"

df = pd.read_sql_query(stmt,engine)
df['action1'] = df.action1.apply(cleanAction)

df.head(25)

In [ ]:
#stmt = "DELETE FROM events"
#conn.execute(stmt)

In [ ]:
#stmt = games.update().values(isProcessed=False)
#conn.execute(stmt)

In [ ]:
conn.close()